# PyOccam Advanced Demo

This notebook demonstrates advanced PyOccam features using **real landslide
susceptibility data** from the Oregon Coast Range, including:

1. Loading and exploring real geospatial data
2. Model selection strategies comparison (BIC vs AIC vs Information)
3. Confusion matrix analysis across models
4. Batch processing patterns
5. CSV conversion for your own data

For basic usage, see `pyoccam_demo.ipynb`

## Setup

In [1]:
import pyoccam
print(f"PyOccam {pyoccam.__version__} - Advanced Demo")

PyOccam 0.9.5 loaded successfully
PyOccam 0.9.5 loaded. Type pyoccam.help() for usage.
PyOccam 0.9.5 - Advanced Demo


## Part 1: Exploring Real Landslide Susceptibility Data

The bundled landslides dataset is derived from GIS layers for the Oregon Coast
Range. It's rich in **categorical variables** like aspect, geology, soil taxonomy,
and land cover. The originally continuous variables — slope, elevation, TWI, and
fault density — have been **binned (reclassified) into discrete categories**, a
common GIS preprocessing step. This makes the dataset ideal for Reconstructability
Analysis, which operates natively on discrete/nominal data rather than requiring
continuous inputs like many other ML methods.

The dependent variable (LS) indicates landslide presence (1) or absence (0).

In [2]:
landslides = pyoccam.load_landslides()
print(f"Dataset: {landslides}")
print(f"Samples: {landslides.n_samples}")
print(f"Features: {landslides.n_features}")
print(f"Target: {landslides.target_name}")
print(f"\nFeature variables:")
for i, name in enumerate(landslides.feature_names):
    print(f"  {i+1:2d}. {name}")

[OK] Loaded landslides: 1077 samples, 20 features, 21 lookup tables
Dataset: OccamData(n_samples=1077, n_features=20, target='LS', lookups=21 vars)
Samples: 1077
Features: 20
Target: LS

Feature variables:
   1. ASpect_reclass
   2. CLay_reclass
   3. CurVature_reclass
   4. ELevation_reclass
   5. Fault_Dns_reclass
   6. HaBitatMap_UTM
   7. Geol_Lith
   8. nlcd_2021_Land_Cover
   9. Geol_rock_Type
  10. Rd_strm_Dns_reclass
  11. SLope_reclass
  12. TWi_reclass
  13. GeomDesc
  14. TaxOrder
  15. TaxSuborder
  16. taxGrtGroup
  17. TaxSubgrp
  18. taxPartSize
  19. TaxClname
  20. DrainageCl


## Part 2: Comparing Model Selection Strategies

OCCAM offers three model selection criteria, each with different tradeoffs:
- **BIC** (most conservative): Penalizes complexity heavily, favors simpler models
- **AIC** (moderate): Less penalty for complexity than BIC
- **Information** (alpha < 0.05): Highest information content with statistical significance

Let's run a comprehensive search on the landslides data and compare all three.

In [4]:
manager = landslides.manager

# Run a comprehensive search
print("Running loopless-up search (levels=7, width=5)...")
report = manager.generate_search_report("full-up", levels=7, width=5)

Running loopless-up search (levels=7, width=5)...
Level 1/7: 20 new models, 5 kept (total: 6)
Level 2/7: 85 new models, 5 kept (total: 11)
Level 3/7: 89 new models, 5 kept (total: 16)
Level 4/7: 90 new models, 5 kept (total: 21)
Level 5/7: 100 new models, 5 kept (total: 26)
Level 6/7: 118 new models, 5 kept (total: 31)
Level 7/7: 130 new models, 5 kept (total: 36)


In [5]:
# Get best models by different criteria
best_bic = manager.get_best_model_by_bic()
best_aic = manager.get_best_model_by_aic()
best_info = manager.get_best_model_by_information()

print("Best models by different criteria:")
print(f"  BIC (most conservative):    {best_bic}")
print(f"  AIC (moderate):             {best_aic}")
print(f"  Information (alpha<0.05):   {best_info}")

Best models by different criteria:
  BIC (most conservative):    IV:CvZ:ElZ:HbZ:GlZ:LcZ:SlZ:TcZ
  AIC (moderate):             IV:ElTwZ:HbZ:LcZ:SlZ:TcZ
  Information (alpha<0.05):   IV:ElTwZ:HbZ:LcZ:SlZ:TcZ


## Part 3: Confusion Matrix Analysis

Compare accuracy, sensitivity, and specificity across the three model
selection strategies, then dive into the detailed confusion matrix.

In [ ]:
print(f"{'Criterion':<10} {'Model':<30} {'Acc':>6} {'Sens':>6} {'Spec':>6}")
print("-" * 70)

for name, model in [("BIC", best_bic), ("AIC", best_aic), ("Info", best_info)]:
    cm = manager.get_confusion_matrix(model, target_state="0")
    if cm.get('has_values', False):
        print(f"{name:<10} {model:<30} "
              f"{cm['train_accuracy']:5.1%} "
              f"{cm['train_sensitivity']:5.1%} "
              f"{cm['train_specificity']:5.1%}")

In [ ]:
# Detailed confusion matrix for the BIC model
print(f"Detailed confusion matrix for BIC model: {best_bic}")
cm = manager.get_confusion_matrix(best_bic, target_state="0")
if cm.get('has_values', False):
    print(f"  TP={cm['train_tp']:4.0f}  FP={cm['train_fp']:4.0f}")
    print(f"  FN={cm['train_fn']:4.0f}  TN={cm['train_tn']:4.0f}")
    print(f"  Precision: {cm['train_precision']:.1%}")
    print(f"  NPV:       {cm['train_npv']:.1%}")

## Part 4: Batch Processing

Process multiple datasets and search configurations systematically.
This pattern is useful for comparing results across different domains.

In [ ]:
datasets = [
    ("Dementia", pyoccam.load_dementia),
    ("Landslides", pyoccam.load_landslides),
]

search_configs = [
    ("loopless-up", 5, 3),
    ("full-up", 4, 3),
]

print(f"{'Dataset':<12} {'Search':<15} {'Best Model':<30} {'Accuracy':<10}")
print("-" * 80)

for ds_name, loader in datasets:
    for search_type, levels, width in search_configs:
        try:
            # Reload data fresh for each search -- the C++ manager
            # cannot be reused across multiple searches
            data = loader()
            mgr = data.manager
            
            report = mgr.generate_search_report(search_type, levels, width)
            best = mgr.get_best_model_by_bic()
            cm = mgr.get_confusion_matrix(best, target_state="0")
            acc = cm.get('train_accuracy', 0.0)
            
            print(f"{ds_name:<12} {search_type:<15} {best:<30} {acc:.1%}")
        except Exception as e:
            print(f"{ds_name:<12} {search_type:<15} ERROR: {e}")

## Part 5: Converting Your Own CSV Data

PyOccam can convert standard CSV files to OCCAM format with optional
train/test splits for model validation.

```python
# Basic conversion (last column = DV)
output_file, data = pyoccam.make_occam_input_from_csv("mydata.csv")

# With train/test split for validation
output_file, data = pyoccam.make_occam_input_from_csv(
    "mydata.csv",
    test_split=0.2,               # 20% held out for testing
    random_state=42,              # Reproducible split
    max_cardinality=20,           # Exclude columns with >20 unique values
    dv_column="target",           # Specify DV column by name
    exclude_columns=["ID", "Name"] # Always exclude these columns
)

# Then analyze
best = data.quick_search()
cm = data.manager.get_confusion_matrix(best, target_state="0")
```

### Command Line Usage

```bash
# Basic conversion with 20% test split
python -m pyoccam csv2occam mydata.csv --test-split 0.2

# With exclusions and custom cardinality
python -m pyoccam csv2occam mydata.csv \
    --test-split 0.3 \
    --random-state 42 \
    --max-cardinality 25 \
    --exclude "ID,x,y,OBJECTID" \
    --dv LS
```

In [ ]:
print("Advanced Demo Complete!")
print("\nFor more info: pyoccam.help()")